In [ ]:
"""Code to format info on movies in a csv file for publication."""

# built-in libraries
import csv
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import pandas as pd
#from sqlalchemy import create_engine

# Create list of dictionaries for each row.
listings = []
scrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-13.csv'
scrape_path = scrapedir + scrapefile
with open(scrape_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            'stars': row[4],
            'times': row[5],
            'format': row[6],
            'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   

# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# Remove some duplicate listings
df = df[df.city != "Belleview, FL"]
df = df[df.city != "High Springs, FL"]
df = df[(df.city != "The Villages, FL") & (df.theater != "Belleview Cinemas")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
times = (df.times.str.split('|', expand=True).rename(columns=lambda x: f"time_{x+1}"))
times = times.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('p', ' p.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('a', ' a.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace(':00', '') if x.dtype == "object" else x)
df = df.drop('times', axis=1)
df = pd.concat([df, times], axis=1, join_axes=[df.index])
df['streetaddy'] = df.addy.str.split(',').str[0]
df.streetaddy = df.streetaddy.str.replace('Blvd', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Blvd..', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Avenue', 'Ave')
df.streetaddy = df.streetaddy.str.replace('Ave', 'Ave.')
df.streetaddy = df.streetaddy.str.replace('Ave..', 'Ave.')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df['cityaddy'] = df.addy.str.split(',').str[1]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('addy', axis=1)

# Ocala market
oc6_df = df.loc[df['theater'] == 'Ocala Center 6']
ocd_dr = df.loc[df['theater'] == 'Ocala Drive-In']
bv_df = df.loc[df['theater'] == 'Belleview Cinemas']
mt_df = df.loc[df['theater'] == 'Marion Theatre']
omp_df = df.loc[df['theater'] == 'Old Mill Playhouse']
regaloca_df = df.loc[df['theater'] == 'Regal Hollywood Stadium 16 & IMAX - Ocala']
rialto_df = df.loc[df['theater'] == 'Rialto Theatre Spanish Springs Town Square']
amc_df = df.loc[df['theater'] == 'AMC Lake Square 12']
barn_df = df.loc[df['theater'] == 'Barnstorm Theater']

# Gainesville market
celeb_df = df.loc[df['theater'] == 'Celebration Pointe 10']
hipp_df = df.loc[df['theater'] == 'Hippodrome State Theatre']
butler_df = df.loc[df['theater'] == 'Regal Butler Town Center 14']
regalgnv_df = df.loc[df['theater'] == 'Regal Royal Park Stadium 16']
uac_df = df.loc[df['theater'] == 'UA Cinema 90 6']
starke_df = df.loc[df['theater'] == 'Florida Twin']


In [ ]:
oc6_df

In [4]:
import csv
from datetime import datetime

scrapedir = '/Users/rayd/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-13.csv'
newfile = scrapedir + scrapefile

def ocadaily_pn(newfile):
    """ Builds print narrative for Ocala daily editions """
    global listings
    listing = []
    listings = []
    ocala_market = 'Ocala, FL'
    gnv_market = ['Gainesville, FL', 'Starke, FL']
    weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
    with open(newfile) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headers = next(csv_reader)
        for row in csv_reader:
            if row[9] in gnv_market:
                row[0] = row[0].split(', FL')[0]
                row[5] = row[5].replace('p | ', ' p.m., ')
                row[5] = row[5].replace('a | ', ' a.m., ')
                row[5] = row[5].replace('0p', '0 p.m.')
                row[5] = row[5].replace('5p', '5 p.m.')
                row[5] = row[5].replace(':00', '')
                row[8] = datetime.strptime(row[8], '%Y-%m-%d').weekday()
                movieday = weekDays[row[8]]
                movie_stuff = f"\"{row[2]}\", {row[3]}, {row[4]}, {row[6]}, " + movieday + f" at {row[5]}\n"
                theater_stuff = f'{row[1]}, {row[0]}, '
                listing = theater_stuff + movie_stuff
                listings.append(listing)
    
    return listings

if __name__ == '__main__':
    ocadaily_pn(newfile)
    print(listings)

['Regal Royal Park, 3702 Newberry Road, Gainesville, "Men in Black: International", PG-13, , Digital 3D, Friday at 12:15 p.m., 9:05 p.m.\n', 'Regal Royal Park, 3702 Newberry Road, Gainesville, "Rocketman", R, 4.5, Standard, Sunday at 12:30 p.m., 3:30 p.m., 6:40 p.m., 9:40 p.m.\n', 'Regal Butler Town Center, 3101 SW 35th Blvd, Gainesville, "Aladdin (2019)", PG, 4.5, Standard, Saturday at 12:30 p.m., 6:45 p.m., 9:55 p.m.\n', 'Regal Butler Town Center, 3101 SW 35th Blvd, Gainesville, "Aladdin (2019)", PG, 4.5, Digital 3D, Saturday at 4:20 p.m.\n', 'Regal Celebration Pointe & RPX, 4901 SW 31st Place, Gainesville, "Rocketman", R, 4.5, Standard, Saturday at 1:20 p.m., 4:40 p.m., 7:50 p.m., 10:45 p.m.\n', 'Regal Butler Town Center, 3101 SW 35th Blvd, Gainesville, "Men in Black: International", PG-13, , Standard, Sunday at 1 p.m., 1:45 p.m., 4:45 p.m., 7 p.m., 7:45 p.m., 10:35 p.m.\n', 'Regal Butler Town Center, 3101 SW 35th Blvd, Gainesville, "Shaft (2019)", R, , Standard, Saturday at 1:15 p.

In [5]:
import csv
from datetime import datetime

scrapedir = '/Users/rayd/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-13.csv'
newfile = scrapedir + scrapefile

def ocadaily_pn(newfile):
    """ Builds print narrative for Ocala daily editions """
    global listings
    listing = []
    listings = []
    ocala_market = 'Ocala, FL'
    gnv_market = ['Gainesville, FL', 'Starke, FL']
    weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
    with open(newfile) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headers = next(csv_reader)
        for row in csv_reader:
            row[0] = row[0].split(', FL')[0]
            row[5] = row[5].replace('p | ', ' p.m., ')
            row[5] = row[5].replace('a | ', ' a.m., ')
            row[5] = row[5].replace('0p', '0 p.m.')
            row[5] = row[5].replace('5p', '5 p.m.')
            row[5] = row[5].replace(':00', '')
            row[8] = datetime.strptime(row[8], '%Y-%m-%d').weekday()
            movieday = weekDays[row[8]]
            movie_stuff = f"\"{row[2]}\", {row[3]}, {row[4]}, {row[6]}, " + movieday + f" at {row[5]}\n"
            theater_stuff = f'{row[1]}, {row[0]}, '
            listing = theater_stuff + movie_stuff
            listings.append(listing)
    
    return listings

if __name__ == '__main__':
    ocadaily_pn(newfile)
    print(listings)

['Regal Butler Town Center, 3101 SW 35th Blvd, Gainesville, "Dark Phoenix", PG-13, 3.5, Standard, Sunday at 1:20 p.m., 3:45 p.m., 7:40 p.m., 10:30 p.m.\n', 'Regal Hollywood & IMAX - Ocala, 2801 SW 27th Avenue, Ocala, "Pokémon Detective Pikachu", PG, 4.5, Standard, Sunday at 3:50 p.m., 6:55 p.m., 9:50 p.m.\n', 'Old Mill Playhouse, 1000 Old Mill Run, The Villages, "The Souvenir", R, 3.5, Standard, Friday at 9:40 a.m., 12 p.m., 2:25 p.m., 8:15 p.m.\n', 'Old Mill Playhouse, 1000 Old Mill Run, The Villages, "Hampstead", PG-13, , Standard, Sunday at 9:35 a.m., 2:40 p.m., 5:45 p.m.\n', 'AMC Lake Square 12, 10401-015 US Hwy. 441 South, Leesburg, "The Secret Life of Pets 2", PG, 4.5, Standard, Saturday at 10 a.m., 10:30 a.m., 11:15 a.m., 1:30 p.m., 2:30 p.m., 3:45 p.m., 6 p.m., 7 p.m., 8:15 p.m.\n', 'Regal Hollywood & IMAX - Ocala, 2801 SW 27th Avenue, Ocala, "Late Night (2019)", R, 4.5, Standard, Saturday at 12:05 p.m., 2:45 p.m., 5:20 p.m., 7:55 p.m., 10:40 p.m.\n', 'Barnstorm Theater, 2720 B

In [ ]:
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()